In [ ]:
import requests

url = 'https://kworb.net/spotify/artists.html'
page = requests.get(url)
page.encoding = 'utf-8'

!pip install bs4

from bs4 import BeautifulSoup

soup = BeautifulSoup(page.text, 'html.parser')

urls = []
a = 0
for link in soup.find_all('a'):
    if '/spotify/artist/' in link.get('href'):
        urls.append(link.get('href'))
        text = link.text
        print(text)

artist_url_map = dict(zip(urls, artist_names))

def GetStats(soup):
    position = 1
    data = []

    for row in soup.select('tbody tr'):

        artist_element = row.select_one('td.text a')
        artist = artist_element.text.strip() if artist_element else None

        streams_element = row.select_one('td:nth-of-type(2)')
        streams = streams_element.text.strip() if streams_element else None

        daily_element = row.select_one('td:nth-of-type(3)')
        daily = daily_element.text.strip() if daily_element else None

        data.append((position, artist, streams, daily))
        position += 1

    return data

stats = GetStats(soup)
display(stats)

import pandas as pd
df = pd.DataFrame(stats)
df.columns = ['Position', 'Artist', 'Streams', 'Daily']
df

def GetStats_2(soup_2):
    data2 = []
    for i in range(3, len(soup_2.select('tbody tr'))):
        row = soup_2.select('tbody tr')[i]

        song_element = row.select_one('td.text a')
        song = song_element.text.strip() if song_element else None

        streams2_element = row.select_one('td:nth-of-type(2)')
        streams2 = streams2_element.text.strip() if streams2_element else None

        daily2_element = row.select_one('td:nth-of-type(3)')
        daily2 = daily2_element.text.strip() if daily2_element else None

        data2.append((song, streams2, daily2))

    return data2

all_artists_song_data = []
for url in urls:
  a = f"https://kworb.net{url}"
  page_2 = requests.get(a)
  page_2.encoding = 'utf-8'

  soup_2 = BeautifulSoup(page_2.text, 'html.parser')
  stats2 = GetStats_2(soup_2)

  current_artist_name = artist_url_map[url]
  songs_with_artist = [(current_artist_name, song_data[0], song_data[1], song_data[2]) for song_data in stats2]

  df_artist = pd.DataFrame(songs_with_artist, columns=['Artist', 'Song', 'Streams', 'Daily'])
  all_artists_song_data.append(df_artist)

all_df = pd.concat(all_artists_song_data, ignore_index=True)
display(all_df)

all_df.to_csv('before_all.csv', index=False)

import pandas as pd
cleaned_df = pd.read_csv('before_all.csv')
cleaned_df.drop_duplicates(subset=['Streams', 'Daily'], inplace=True)

cleaned_df['Streams'] = cleaned_df['Streams'].astype(str).str.replace(',', '', regex=False).astype(int)
top = cleaned_df.sort_values('Streams', ascending=False).head(10000)

display(top)

!pip install pylast

df_scraped = pd.read_csv('top_artists_songs (1).csv')
display(df_scraped.head())

import pylast
import time
import pandas as pd
from google.colab import userdata

LASTFM_API_KEY = userdata.get('LASTFM_API_KEY')



lastfm_data = []
if 'df_scraped' in locals() and isinstance(df_scraped, pd.DataFrame) and not df_scraped.empty:
    if 'Artist' in df_scraped.columns and 'Song' in df_scraped.columns:
        rows_to_process = df_scraped.itertuples()

    for index, row in enumerate(rows_to_process):
        artist_name = row.Artist
        track_title = row.Song
        print(f"Обрабатываем трек {index + 1}: '{track_title}' от '{artist_name}'")

        try:
                    track = network.get_track(artist_name, track_title)

                    duration_ms = None
                    try:
                         duration_ms = track.get_duration()
                         if duration_ms == '0':
                             duration_ms = None
                         else:
                             duration_ms = int(duration_ms)
                    except:
                        pass

                    genres = []
                    try:
                        tags = track.get_top_tags(limit=10)
                        if tags:
                            genres = [tag.item.get_name() for tag in tags]
                    except:
                        pass


                    lastfm_data.append({
                        'scraped_artist': artist_name,
                        'scraped_song': track_title,
                        'lastfm_duration_ms': duration_ms,
                        'lastfm_genres': ", ".join(genres) if genres else None,
                        'lastfm_playcount': track.get_playcount() if hasattr(track, 'get_playcount') else None
                    })

                except pylast.WSError as e:
                    print(f"Ошибка Last.fm API для '{track_title}' by '{artist_name}': {e}")
                    lastfm_data.append({
                        'scraped_artist': artist_name,
                        'scraped_song': track_title,
                        'lastfm_duration_ms': None,
                        'lastfm_genres': None,
                        'lastfm_playcount': None
                    })
                except Exception as e:
                    print(f"Произошла другая ошибка для '{track_title}' by '{artist_name}': {e}")
                    lastfm_data.append({
                        'scraped_artist': artist_name,
                        'scraped_song': track_title,
                        'lastfm_duration_ms': None,
                        'lastfm_genres': None,
                        'lastfm_playcount': None
                    })

                time.sleep(0.2) 

            df_lastfm = pd.DataFrame(lastfm_data)
            print("\nПредварительный просмотр данных из Last.fm:")
            display(df_lastfm.head())
            print(f"\nВсего собрано записей из Last.fm: {len(df_lastfm)}")

        else:
            print("DataFrame df_scraped не содержит необходимые колонки 'Artist' и/или 'Song'.")
    else:
        print("DataFrame df_scraped не загружен или пуст. Пожалуйста, загрузите данные скрапинга.»)

!pip install pylast

import pandas as pd
df_scraped = pd.read_csv('top_artists_songs.csv')
display(df_scraped.head())

import pylast
import time
import pandas as pd
from google.colab import userdata

LASTFM_API_KEY = userdata.get('LASTFM_API_KEY')

lastfm_data = []

if 'df_scraped' in locals() and isinstance(df_scraped, pd.DataFrame) and not df_scraped.empty:
    if 'Artist' in df_scraped.columns and 'Song' in df_scraped.columns:
        rows_to_process = df_scraped.itertuples()

        for index, row in enumerate(rows_to_process):
            artist_name = row.Artist
            track_title = row.Song

            duration_ms = None
            genres = []
            playcount = None

            try:
                track = network.get_track(artist_name, track_title)

                try:
                    duration_ms = track.get_duration()
                    if duration_ms == '0':
                        duration_ms = None
                    else:
                        duration_ms = int(duration_ms)
                except Exception:
                    pass 

                try:
                    tags = track.get_top_tags(limit=10)
                    if tags:
                        genres = [tag.item.get_name() for tag in tags]
                except Exception:
                    pass 

                playcount = track.get_playcount() if hasattr(track, 'get_playcount') else None

            finally:
                lastfm_data.append({
                    'scraped_artist': artist_name,
                    'scraped_song': track_title,
                    'lastfm_duration_ms': duration_ms,
                    'lastfm_genres': ", ".join(genres) if genres else None,
                    'lastfm_playcount': playcount
                })

            time.sleep(0.2)

        df_lastfm = pd.DataFrame(lastfm_data)
        display(df_lastfm.head())
    else:
        print("No information")
else:
    print("Error!")

df_lastfm.to_csv('lastfm_data_filtered.csv', index=False)





#---------------------2 часть. Графики---------------------
import pandas as pd

df_lastfm_original = pd.read_csv('/content/lastfm_data.csv')

print("DataFrame 'df_lastfm_original' loaded successfully. Displaying the first 5 rows:")
display(df_lastfm_original.head())

print("\nИнформация о столбцах 'df_lastfm_original':")
df_lastfm_original.info()

import re

df_lastfm_processed = df_lastfm_original.copy()
df_lastfm_processed.dropna(inplace=True)
df_lastfm = df_lastfm_processed.copy()
def extract_year(genres_string):
    if isinstance(genres_string, str):
        years = re.findall(r'\b(19\d{2}|20\d{2})\b', genres_string)
        if years:
            return int(years[0])
    return None

df_lastfm_processed['release_year'] = df_lastfm_processed['lastfm_genres'].apply(extract_year)
df_lastfm_processed.dropna(subset=['release_year'], inplace=True)
df_lastfm_processed['release_year'] = df_lastfm_processed['release_year'].astype(int)

display(df_lastfm_processed.head())

import pandas as pd
import re

preferred_genres = [
    'Pop', 'Rock', 'Folk', 'Reggae', 'Latin', 'Country', 'Gospel',
    'Metal', 'Jazz', 'Blues', 'Punk', 'Classical'
]
preferred_genres_lower = [g.lower() for g in preferred_genres]

def categorize_genre(genres_string):
    if pd.isna(genres_string) or str(genres_string).lower() == 'nan':
        return None

    genres_string_lower = str(genres_string).lower()

    for pg_lower in preferred_genres_lower:
        if re.search(r'\b' + re.escape(pg_lower) + r'\b', genres_string_lower):
            return preferred_genres[preferred_genres_lower.index(pg_lower)]

    first_word = None
    parts = re.split(r'[,\s]+', genres_string.strip())
    for part in parts:
        if part.strip() and part.lower() not in ['and', 'with', 'feat', 'ft', 'the']:
            first_word = part.strip()
            break

    return first_word.capitalize() if first_word else None

df_lastfm['cleaned_genre'] = df_lastfm['lastfm_genres'].apply(categorize_genre)

display(df_lastfm.head())

import pandas as pd
import re

preferred_genres = [
    'Pop', 'Rock', 'Folk', 'Reggae', 'Latin', 'Country', 'Gospel',
    'Metal', 'Jazz', 'Blues', 'Punk', 'Classical'
]
preferred_genres_lower = [g.lower() for g in preferred_genres]

def categorize_genre(genres_string):
    if pd.isna(genres_string) or str(genres_string).lower() == 'nan': 
        return None

    genres_string_lower = str(genres_string).lower() 

    for pg_lower in preferred_genres_lower:
        if re.search(r'\b' + re.escape(pg_lower) + r'\b', genres_string_lower):
            return preferred_genres[preferred_genres_lower.index(pg_lower)]

    first_word = None
    parts = re.split(r'[,\s]+', genres_string.strip())
    for part in parts:
        if part.strip() and part.lower() not in ['and', 'with', 'feat', 'ft', 'the']:# Filter out common conjunctions/features
            first_word = part.strip()
            break

    return first_word.capitalize() if first_word else None

df_lastfm['cleaned_genre'] = df_lastfm['lastfm_genres'].apply(categorize_genre)

display(df_lastfm.head())

df_top_artists = pd.read_csv('/content/top_artists_songs (3).csv')
df_top_artists.head()

df_merged = pd.merge(
    df_top_artists,
    df_lastfm[['scraped_artist', 'scraped_song', 'lastfm_genres', 'lastfm_duration_ms']],
    how='left',
    left_on=['Artist', 'Song'],
    right_on=['scraped_artist', 'scraped_song']
)

df_merged = df_merged.drop(columns=['scraped_artist', 'scraped_song'], errors='ignore')

print("DataFrame 'df_merged' created successfully. Displaying the first 5 rows:")
df_merged.head()

df_merged['Song_Number'] = df_merged.index + 1
df_merged = df_merged[['Song_Number', 'Artist', 'Song', 'Streams', 'Daily', 'lastfm_genres', 'lastfm_duration_ms']]

display(df_merged.head())

import pandas as pd

# Start with df_top_artists as the base
# This ensures df_merged is rebuilt from a known good state
df_merged = df_top_artists.copy()

# Merge cleaned_genre and lastfm_duration_ms from df_lastfm
df_merged = pd.merge(
    df_merged,
    df_lastfm[['scraped_artist', 'scraped_song', 'cleaned_genre', 'lastfm_duration_ms']],
    how='left',
    left_on=['Artist', 'Song'],
    right_on=['scraped_artist', 'scraped_song']
)

df_merged = df_merged.drop(columns=['scraped_artist', 'scraped_song'], errors='ignore')
# Rename cleaned_genre to lastfm_genres for compatibility with existing plot code
df_merged = df_merged.rename(columns={'cleaned_genre': 'lastfm_genres'})

# Filter out tracks with duration 0 or less
df_merged = df_merged[df_merged['lastfm_duration_ms'] > 0].copy()

df_merged['Song_Number'] = df_merged.index + 1
df_merged = df_merged[['Song_Number', 'Artist', 'Song', 'Streams', 'Daily', 'lastfm_genres', 'lastfm_duration_ms']]

# Ensure clean state by dropping old release_year columns before adding new one
columns_to_drop_before_merge = ['release_year', 'release_year_x', 'release_year_y']
for col in columns_to_drop_before_merge:
    if col in df_merged.columns:
        df_merged = df_merged.drop(columns=[col])

df_merged = pd.merge(
    df_merged,
    df_lastfm_processed[['scraped_artist', 'scraped_song', 'release_year']],
    how='left',
    left_on=['Artist', 'Song'],
    right_on=['scraped_artist', 'scraped_song']
)

df_merged = df_merged.drop(columns=['scraped_artist', 'scraped_song'], errors='ignore')
df_merged['release_year'] = df_merged['release_year'].astype('Int64')

df_merged['lastfm_duration_s'] = df_merged['lastfm_duration_ms'] / 1000

df_merged.loc[
    (df_merged['Artist'] == 'Queen') &
    (df_merged['Song'].str.contains('Somebody To Love', na=False)),
    'release_year'
] = 1976

display(df_merged.head())

import matplotlib.pyplot as plt
import seaborn as sns

artist_counts = df_merged['Artist'].value_counts()

top_10_artists = artist_counts.head(10)

plt.figure(figsize=(12, 7))
sns.barplot(x=top_10_artists.index, y=top_10_artists.values, palette='viridis')
plt.title('Топ-10 артистов по количеству песен в датасете')
plt.xlabel('Артист')
plt.ylabel('Количество песен')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

import matplotlib.pyplot as plt

genre_counts = df_merged['lastfm_genres'].value_counts().dropna()
genres_to_display = ['Pop', 'Rock', 'Rap', 'Hip-hop', 'Rnb', 'Jazz', 'Folk']
genres_for_other = [g for g in genre_counts.index if g not in genres_to_display]
other_genres_sum = genre_counts[genres_for_other].sum()
genre_counts_filtered = genre_counts[genre_counts.index.isin(genres_to_display)].copy()

if other_genres_sum > 0:
    genre_counts_filtered['Other'] = other_genres_sum

genre_counts_filtered = genre_counts_filtered.sort_values(ascending=False)

plt.figure(figsize=(10, 8))
plt.pie(genre_counts_filtered, labels=genre_counts_filtered.index, autopct=lambda p: '{:.1f}%'.format(p) if p > 0 else '', startangle=140, textprops={'fontsize': 7})
plt.title('Распределение песен по жанрам (Pop, Rock, Rap, Hip-hop, Rnb, Jazz, Folk и Other)')
plt.show()

import matplotlib.pyplot as plt
import seaborn as sns

df_merged['Streams'] = df_merged['Streams'].astype(str).str.replace(',', '', regex=False).astype(float)
df_merged['lastfm_duration_s'] = df_merged['lastfm_duration_ms'] / 1000
df_plot = df_merged.dropna(subset=['lastfm_duration_s', 'Streams', 'lastfm_genres'])
genres_to_display = ['Pop', 'Rock', 'Rap', 'Hip-hop', 'Rnb', 'Jazz', 'Folk']

df_plot['plot_genre'] = df_plot['lastfm_genres'].apply(
    lambda x: x if x in genres_to_display else 'Other'
)

custom_palette = [
    'lime',        # салатовый
    'red',         # ярко-красный
    'darkorange',  # ярко-оранжевый
    'darkgoldenrod', # тёмно-жёлтый
    'darkgreen',   # тёмно-зелёный
    'blue',        # ярко-синий
    'fuchsia',     # ярко-фиолетовый
    'hotpink'      # ярко-розовый
]

plt.figure(figsize=(12, 8))
sns.scatterplot(x='lastfm_duration_s', y='Streams', hue='plot_genre', data=df_plot, alpha=0.6, palette=custom_palette)
plt.title('Зависимость прослушиваний трека от его длины по жанра')
plt.xlabel('Длительность трека (с)')
plt.ylabel('Количество прослушиваний')
plt.grid(True)
plt.legend(title='Жанр', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout() 
plt.show()

df_merged['Streams'] = df_merged['Streams'].astype(str).str.replace(',', '', regex=False).astype(float)
df_merged['lastfm_duration_s'] = df_merged['lastfm_duration_ms'] / 1000
df_plot = df_merged.dropna(subset=['lastfm_duration_s', 'Streams', 'lastfm_genres'])

df_plot_only_pop = df_plot[df_plot['lastfm_genres'] == 'Pop']



plt.figure(figsize=(12, 8))
sns.scatterplot(x='lastfm_duration_s', y='Streams', hue='lastfm_genres', data=df_plot_only_pop, alpha=1)
plt.title('Зависимость прослушиваний поп-трека от его длины')
plt.xlabel('Длительность трека (с)')
plt.ylabel('Количество прослушиваний')
plt.grid(True)
plt.legend(title='Жанр', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout() 
plt.show()

df_merged['Streams'] = df_merged['Streams'].astype(str).str.replace(',', '', regex=False).astype(float)
df_merged['lastfm_duration_s'] = df_merged['lastfm_duration_ms'] / 1000
df_plot = df_merged.dropna(subset=['lastfm_duration_s', 'Streams', 'lastfm_genres'])

df_plot_only_rock = df_plot[df_plot['lastfm_genres'] == 'Rock']



plt.figure(figsize=(12, 8))
sns.scatterplot(x='lastfm_duration_s', y='Streams', hue='lastfm_genres', data=df_plot_only_rock, alpha=1)
plt.title('Зависимость прослушиваний рок-трека от его длины')
plt.xlabel('Длительность трека (с)')
plt.ylabel('Количество прослушиваний')
plt.grid(True)
plt.legend(title='Жанр', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout() 
plt.show()

import matplotlib.pyplot as plt
import seaborn as sns

df_merged['Streams'] = df_merged['Streams'].astype(str).str.replace(',', '', regex=False).astype(float)
df_merged['Daily'] = df_merged['Daily'].astype(str).str.replace(',', '', regex=False).astype(float)

df_merged['lastfm_duration_s'] = df_merged['lastfm_duration_ms'] / 1000
df_plot = df_merged.dropna(subset=['lastfm_duration_s', 'Daily', 'lastfm_genres']).copy()
genres_to_display = ['Pop', 'Rock', 'Rap', 'Hip-hop', 'Rnb', 'Jazz', 'Folk']

df_plot['plot_genre'] = df_plot['lastfm_genres'].apply(
    lambda x: x if x in genres_to_display else 'Other'
)

custom_palette = [
    'lime',        # салатовый
    'red',         # ярко-красный
    'darkorange',  # ярко-оранжевый
    'darkgoldenrod', # тёмно-жёлтый
    'darkgreen',   # тёмно-зелёный
    'blue',        # ярко-синий
    'fuchsia',     # ярко-фиолетовый
    'hotpink'      # ярко-розовый
]

plt.figure(figsize=(12, 8))
sns.scatterplot(x='lastfm_duration_s', y='Daily', hue='plot_genre', data=df_plot, alpha=0.6, palette=custom_palette)
plt.title('Зависимость дневных прослушиваний от длины трека по жанрам')
plt.xlabel('Длительность трека (с)')
plt.ylabel('Количество дневных прослушиваний')
plt.grid(True)
plt.legend(title='Жанр', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

df_merged['Streams'] = df_merged['Streams'].astype(str).str.replace(',', '', regex=False).astype(float)
df_merged['Daily'] = df_merged['Daily'].astype(str).str.replace(',', '', regex=False)

df_plot_year_streams = df_merged.dropna(subset=['release_year', 'Streams', 'lastfm_genres']).copy()
genres_to_display = ['Pop', 'Rock', 'Rap', 'Hip-hop', 'Rnb', 'Jazz', 'Folk']

df_plot_year_streams['plot_genre'] = df_plot_year_streams['lastfm_genres'].apply(
    lambda x: x if x in genres_to_display else 'Other'
)

custom_palette = [
    'lime',        # салатовый
    'red',         # ярко-красный
    'darkorange',  # ярко-оранжевый
    'darkgoldenrod', # тёмно-жёлтый
    'darkgreen',   # тёмно-зелёный
    'blue',        # ярко-синий
    'fuchsia',     # ярко-фиолетовый
    'hotpink'      # ярко-розовый
]

plt.figure(figsize=(14, 8))
sns.scatterplot(x='release_year', y='Streams', hue='plot_genre', data=df_plot_year_streams, alpha=0.6, palette=custom_palette)
plt.title('Зависимость прослушиваний от года выпуска по жанрам')
plt.xlabel('Год выпуска')
plt.ylabel('Количество прослушиваний')
plt.grid(True)
plt.legend(title='Жанр', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

import matplotlib.pyplot as plt
import seaborn as sns

df_merged['Streams'] = df_merged['Streams'].astype(str).str.replace(',', '', regex=False).astype(float)
df_merged['Daily'] = df_merged['Daily'].astype(str).str.replace(',', '', regex=False).astype(float)

df_merged['lastfm_duration_s'] = df_merged['lastfm_duration_ms'] / 1000
# Исправляем SettingWithCopyWarning, явно создавая копию
df_plot_daily_year = df_merged.dropna(subset=['release_year', 'Daily', 'lastfm_genres']).copy()

# Define the genres to display individually
genres_to_display = ['Pop', 'Rock', 'Rap', 'Hip-hop', 'Rnb', 'Jazz', 'Folk']

# Create a temporary column for plotting genres
df_plot_daily_year['plot_genre'] = df_plot_daily_year['lastfm_genres'].apply(
    lambda x: x if x in genres_to_display else 'Other'
)

custom_palette = [
    'lime',        # салатовый
    'red',         # ярко-красный
    'darkorange',  # ярко-оранжевый
    'darkgoldenrod', # тёмно-жёлтый
    'darkgreen',   # тёмно-зелёный
    'blue',        # ярко-синий
    'fuchsia',     # ярко-фиолетовый
    'hotpink'      # ярко-розовый
]

plt.figure(figsize=(14, 8))
sns.scatterplot(x='release_year', y='Daily', hue='plot_genre', data=df_plot_daily_year, alpha=0.6, palette=custom_palette)
plt.title('Зависимость дневных прослушиваний от года выпуска по жанрам')
plt.xlabel('Год выпуска')
plt.ylabel('Количество дневных прослушиваний')
plt.grid(True)
plt.legend(title='Жанр', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

df_merged['Streams'] = df_merged['Streams'].astype(str).str.replace(',', '', regex=False).astype(float)
df_old_tracks = df_merged[df_merged['release_year'] < 2014].dropna(subset=['Streams', 'release_year'])
evergreen_classics = pd.DataFrame()

for year, group in df_old_tracks.groupby('release_year'):
    top_tracks_for_year = group.sort_values(by='Streams', ascending=False).head(5)
    evergreen_classics = pd.concat([evergreen_classics, top_tracks_for_year])

evergreen_classics = evergreen_classics.reset_index(drop=True).sort_values(by=['release_year', 'Streams'], ascending=[True, False])

print("Датасет 'Вечнозелёная классика' создан. Первые 10 строк:")
display(evergreen_classics.head(10))

print("\nИнформация о датасете 'Вечнозелёная классика':")
evergreen_classics.info()

import matplotlib.pyplot as plt
import seaborn as sns

df_recent_songs = df_merged[df_merged['release_year'] >= 2023].copy()
artists_in_recent_years = df_recent_songs['Artist'].unique()

rising_stars_artists = []
for artist in artists_in_recent_years:
    if df_merged[df_merged['Artist'] == artist]['release_year'].min() >= 2023:
        rising_stars_artists.append(artist)

df_rising_stars = df_recent_songs[df_recent_songs['Artist'].isin(rising_stars_artists)]

rising_stars_counts = df_rising_stars['Artist'].value_counts()

top_5_rising_stars = rising_stars_counts.head(5)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_5_rising_stars.index, y=top_5_rising_stars.values, palette='viridis')
plt.title('Восходящие звёзды (Топ-5 артистов с песнями не раньше 2023 года)')
plt.xlabel('Артист')
plt.ylabel('Количество песен в датасете (выпущенных после 2023 года)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("Топ-5 Восходящих звёзд:")
display(top_5_rising_stars)






